In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Accelerating HuggingFace T5 Inference with TensorRT

T5 is an encoder-decoder model that converts all NLP problems into a text-to-text format. More specifically, it does so by encoding  different tasks as text directives in the input stream. This enables a single model to be trained supervised on a wide variety of NLP tasks such as translation, classification, Q&A and summarization.

This notebook shows 3 easy steps to convert a [HuggingFace PyTorch T5 model](https://huggingface.co/transformers/model_doc/t5.html) to a TensorRT engine for high-performance inference.

1. [Download HuggingFace T5 model](#1)
1. [Convert to ONNX format](#2)
1. [Convert to TensorRT engine](#3)

## Prerequisite

Follow the instruction at https://github.com/NVIDIA/TensorRT to build the TensorRT-OSS docker container required to run this notebook.

Next, we install some extra dependencies, then restart the kernel.

In [ ]:
%%capture
!pip3 install -r ../requirements.txt

# install Pytorch with A100 support
!pip3 install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio===0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

import IPython
import time
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

time.sleep(10)

In [1]:
import os
import sys
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)

import torch
import tensorrt as trt

# huggingface
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    T5Config,
)

# to display detailed TensorRT conversion process
from NNDF.logger import G_LOGGER
G_LOGGER.setLevel(level=G_LOGGER.DEBUG)

2021-11-12 16:11:39.555991: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


<a id="1"></a>

## 1. Download HuggingFace T5 model

First, we download the original HuggingFace PyTorch T5 model from HuggingFace model hubs, together with its associated tokernizer.

In [2]:
# The T5 variants  that are suported by TensorRT 8 are:  t5-small (60M), t5-base (220M), t5-large (770M). However, as the conversion process takes long time with
# the base and large models, we recommend using the ../run.py script. See ../README.md for more details.
T5_VARIANT = 't5-small'

t5_model = T5ForConditionalGeneration.from_pretrained(T5_VARIANT)
tokenizer = T5Tokenizer.from_pretrained(T5_VARIANT)
config = T5Config(T5_VARIANT)

In [3]:
# save model locally
pytorch_model_dir = './models/{}/pytorch'.format(T5_VARIANT)
!mkdir -p $pytorch_model_dir

t5_model.save_pretrained(pytorch_model_dir)
print("Pytorch Model saved to {}".format(pytorch_model_dir))

Pytorch Model saved to ./models/t5-small/pytorch


### Inference with PyTorch model

Next, we will carry out inference with the PyTorch model.

#### Single example inference

In [4]:
inputs = tokenizer("translate English to German: This is good.", return_tensors="pt")

# inference on a single example
t5_model.eval()
with torch.no_grad():
    outputs = t5_model(**inputs, labels=inputs["input_ids"])

logits = outputs.logits

In [5]:
# Generate sequence for an input
outputs = t5_model.to('cuda:0').generate(inputs.input_ids.to('cuda:0'))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Das ist gut.


#### Model inference benchmark: encoder and decoder stacks

For benchmarking purposes, we will employ a helper functions `encoder_inference` and `decoder_inference` which execute the inference repeatedly for the T5 encoder and decoder stacks separately, and measure end to end execution time. Let's take note of this execution time for comparison with TensorRT. 
 
`TimingProfile` is a named tuple that specifies the number of experiments and number of times to call the function per iteration (and number of warm-up calls although it is not used here).

In [6]:
from T5.measurements import decoder_inference, encoder_inference, full_inference_greedy
from T5.export import T5EncoderTorchFile, T5DecoderTorchFile
from NNDF.networks import TimingProfile

t5_torch_encoder = T5EncoderTorchFile.TorchModule(t5_model.encoder)
t5_torch_decoder = T5DecoderTorchFile.TorchModule(
    t5_model.decoder, t5_model.lm_head, t5_model.config
)

In [7]:
input_ids = inputs.input_ids

encoder_last_hidden_state, encoder_e2e_median_time = encoder_inference(
    t5_torch_encoder, input_ids, TimingProfile(iterations=10, number=1, warmup=1)
)
encoder_e2e_median_time

[2021-11-12 16:11:47,705][OSS][DEBUG] Measuring inference call with warmup: 3 and number: 1 and iterations 10
[2021-11-12 16:11:47,717][OSS][DEBUG] Warmup times: [0.004397520329803228, 0.0036489246413111687, 0.00367508502677083]


0.003571802983060479

In [8]:
_, decoder_e2e_median_time = decoder_inference(
    t5_torch_decoder, input_ids, encoder_last_hidden_state, TimingProfile(iterations=10, number=1, warmup=1)
)
decoder_e2e_median_time

[2021-11-12 16:11:47,765][OSS][DEBUG] Measuring inference call with warmup: 3 and number: 1 and iterations 10
[2021-11-12 16:11:47,784][OSS][DEBUG] Warmup times: [0.006626531947404146, 0.005857839249074459, 0.005829236004501581]


0.005843587452545762

#### Full model inference and benchmark

Next, we will try the T5 model for the task of translation from English to German.

For benchmarking purposes, we will employ a helper function `full_inference_greedy` which executes the inference repeatedly and measures end to end execution time. Let's take note of this execution time for comparison with TensorRT. 

In [9]:
from T5.T5ModelConfig import T5ModelTRTConfig

decoder_output_greedy, full_e2e_median_runtime = full_inference_greedy(
    t5_torch_encoder,
    t5_torch_decoder,
    input_ids,
    tokenizer,
    TimingProfile(iterations=10, number=1, warmup=1),
    max_length=T5ModelTRTConfig.MAX_SEQUENCE_LENGTH[T5_VARIANT],
)
full_e2e_median_runtime

[2021-11-12 16:11:47,854][OSS][DEBUG] Measuring inference call with warmup: 3 and number: 1 and iterations 10
[2021-11-12 16:11:47,950][OSS][DEBUG] Warmup times: [0.032644343096762896, 0.031276879366487265, 0.031249647960066795]


0.03125189268030226

Let us decode the model's output back into text.

In [10]:
# De-tokenize output to raw text
print(tokenizer.decode(decoder_output_greedy[0], skip_special_tokens=True))

Das ist gut.


<a id="2"></a>

## 2. Convert to ONNX

Prior to converting the model to a TensorRT engine, we will first convert the PyTorch model to an intermediate universal format.

ONNX is an open format for machine learning and deep learning models. It allows you to convert deep learning and machine learning models from different frameworks such as TensorFlow, PyTorch, MATLAB, Caffe, and Keras to a single format.

The steps to convert a PyTorch model to TensorRT are as follows:
- Convert the pretrained image segmentation PyTorch model into ONNX.
- Import the ONNX model into TensorRT.
- Apply optimizations and generate an engine.
- Perform inference on the GPU. 

For the T5 model, we will convert the encoder and decoder seperately.

In [11]:
# helpers
from NNDF.networks import NetworkMetadata, Precision
from T5.T5ModelConfig import T5Metadata

In [12]:
onnx_model_path = './models/{}/ONNX'.format(T5_VARIANT)
!mkdir -p $onnx_model_path

metadata=NetworkMetadata(T5_VARIANT, precision=Precision(fp16=False), other=T5Metadata(kv_cache=False))

encoder_onnx_model_fpath = T5_VARIANT + "-encoder.onnx"
decoder_onnx_model_fpath = T5_VARIANT + "-decoder-with-lm-head.onnx"

t5_encoder = T5EncoderTorchFile(t5_model.to('cpu'), metadata)
t5_decoder = T5DecoderTorchFile(t5_model.to('cpu'), metadata)

onnx_t5_encoder = t5_encoder.as_onnx_model(
    os.path.join(onnx_model_path, encoder_onnx_model_fpath), force_overwrite=False
)
onnx_t5_decoder = t5_decoder.as_onnx_model(
    os.path.join(onnx_model_path, decoder_onnx_model_fpath), force_overwrite=False
)

/home/trtuser/.local/lib/python3.8/site-packages/torch/onnx/utils.py:298: UserWarning: It is recommended that constant folding be turned off ('do_constant_folding=False') when exporting the model in training-amenable mode, i.e. with 'training=TrainingMode.TRAIN' or 'training=TrainingMode.PRESERVE' (when model is in training mode). Otherwise, some learnable model parameters may not translate correctly in the exported ONNX model because constant folding mutates model parameters. Please consider turning off constant folding or setting the training=TrainingMode.EVAL.
  warnings.warn("It is recommended that constant folding be turned off ('do_constant_folding=False') "
/home/trtuser/.local/lib/python3.8/site-packages/transformers/models/t5/modeling_t5.py:428: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might no

In [13]:
del t5_model, onnx_t5_encoder, onnx_t5_decoder

<a id="3"></a>

## 3. Convert to TensorRT

Now we are ready to parse the ONNX encoder and decoder models and convert them to optimized TensorRT engines.

**Note:** As TensorRT carries out many optimizations, this conversion process might take a while.

In [14]:
from T5.export import T5DecoderONNXFile, T5EncoderONNXFile

In [15]:
tensorrt_model_path = './models/{}/tensorrt'.format(T5_VARIANT)
!rm -rf $tensorrt_model_path
!mkdir -p $tensorrt_model_path

In [16]:
t5_trt_encoder_engine = T5EncoderONNXFile(
                os.path.join(onnx_model_path, encoder_onnx_model_fpath), metadata
            ).as_trt_engine(os.path.join(tensorrt_model_path, encoder_onnx_model_fpath) + ".engine")

[W] 'colored' module is not installed, will not use colors when logging. To enable colors, please install the 'colored' module: python3 -m pip install colored
[V] Loaded Module: tensorrt           | Version: 8.2.0.6  | Path: ['/usr/lib/python3.8/dist-packages/tensorrt']
[11/12/2021-16:11:53] [TRT] [I] [MemUsageChange] Init CUDA: CPU +435, GPU +0, now: CPU 4233, GPU 2230 (MiB)
[11/12/2021-16:11:53] [TRT] [I] ----------------------------------------------------------------
[11/12/2021-16:11:53] [TRT] [I] Input filename:   ./models/t5-small/ONNX/t5-small-encoder.onnx
[11/12/2021-16:11:53] [TRT] [I] ONNX IR version:  0.0.6
[11/12/2021-16:11:53] [TRT] [I] Opset version:    12
[11/12/2021-16:11:53] [TRT] [I] Producer name:    pytorch
[11/12/2021-16:11:53] [TRT] [I] Producer version: 1.9
[11/12/2021-16:11:53] [TRT] [I] Domain:           
[11/12/2021-16:11:53] [TRT] [I] Model version:    0
[11/12/2021-16:11:53] [TRT] [I] Doc string:       
[11/12/2021-16:11:53] [TRT] [I] ----------------------

In [17]:
t5_trt_decoder_engine = T5DecoderONNXFile(
                os.path.join(onnx_model_path, decoder_onnx_model_fpath), metadata
            ).as_trt_engine(os.path.join(tensorrt_model_path, decoder_onnx_model_fpath) + ".engine")

[11/12/2021-16:13:08] [TRT] [I] [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 5585, GPU 2978 (MiB)
[11/12/2021-16:13:08] [TRT] [I] ----------------------------------------------------------------
[11/12/2021-16:13:08] [TRT] [I] Input filename:   ./models/t5-small/ONNX/t5-small-decoder-with-lm-head.onnx
[11/12/2021-16:13:08] [TRT] [I] ONNX IR version:  0.0.6
[11/12/2021-16:13:08] [TRT] [I] Opset version:    12
[11/12/2021-16:13:08] [TRT] [I] Producer name:    pytorch
[11/12/2021-16:13:08] [TRT] [I] Producer version: 1.9
[11/12/2021-16:13:08] [TRT] [I] Domain:           
[11/12/2021-16:13:08] [TRT] [I] Model version:    0
[11/12/2021-16:13:08] [TRT] [I] Doc string:       
[11/12/2021-16:13:08] [TRT] [I] ----------------------------------------------------------------
[11/12/2021-16:13:08] [TRT] [V] Plugin creator already registered - ::GridAnchor_TRT version 1
[11/12/2021-16:13:08] [TRT] [V] Plugin creator already registered - ::GridAnchorRect_TRT version 1
[11/12/2021-16:13:08] [

### Inference with TensorRT engine

Great, if you have reached this stage, it means we now have an optimized TensorRT engine for the T5 model, ready for us to carry out inference. 

#### Single example inference
The T5 model with TensorRT backend can now be employed in place of the original HuggingFace T5 model.


In [18]:
# Initialize TensorRT engines
from T5.trt import T5TRTEncoder, T5TRTDecoder

tfm_config = T5Config(
    use_cache=True,
    num_layers=T5ModelTRTConfig.NUMBER_OF_LAYERS[T5_VARIANT],
)
    
t5_trt_encoder = T5TRTEncoder(
                t5_trt_encoder_engine, metadata, tfm_config
            )
t5_trt_decoder = T5TRTDecoder(
                t5_trt_decoder_engine, metadata, tfm_config
            )

[2021-11-12 16:15:35,521][OSS][INFO] Reading and loading engine file ./models/t5-small/tensorrt/t5-small-encoder.onnx.engine using trt native runner.
[2021-11-12 16:15:36,504][OSS][DEBUG] Number of profiles detected in engine: 2
[2021-11-12 16:15:36,505][OSS][DEBUG] Selected profile: [(1, 1), (1, 256), (1, 512)]
[2021-11-12 16:15:36,506][OSS][INFO] Reading and loading engine file ./models/t5-small/tensorrt/t5-small-decoder-with-lm-head.onnx.engine using trt native runner.
[2021-11-12 16:15:37,802][OSS][DEBUG] Number of profiles detected in engine: 3
[2021-11-12 16:15:37,803][OSS][DEBUG] Selected profile: [(1, 1), (1, 256), (1, 512)]


In [19]:
# Inference on a single sample
encoder_last_hidden_state = t5_trt_encoder(input_ids=input_ids)
outputs = t5_trt_decoder(input_ids, encoder_last_hidden_state)

In [20]:
# Generate sequence for an input
from transformers.generation_stopping_criteria import (
    MaxLengthCriteria,
    StoppingCriteriaList,
)

max_length = 64

decoder_input_ids = torch.full(
    (1, 1), tokenizer.convert_tokens_to_ids(tokenizer.pad_token), dtype=torch.int32
)

encoder_last_hidden_state = t5_trt_encoder(input_ids=input_ids)

outputs = t5_trt_decoder.greedy_search(
            input_ids=decoder_input_ids,
            encoder_hidden_states=encoder_last_hidden_state,
            stopping_criteria = StoppingCriteriaList([MaxLengthCriteria(max_length)])
        )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Das ist gut.


In [21]:
outputs

tensor([[   0,  644,  229, 1806,    5,    1]])

#### TRT engine inference benchmark: encoder and decoder stacks
First, we will bechmark the encoder and decoder stacks as before.

In [22]:
encoder_last_hidden_state, encoder_e2e_median_time = encoder_inference(
    t5_trt_encoder, input_ids, TimingProfile(iterations=10, number=1, warmup=1),
)
encoder_e2e_median_time


[2021-11-12 16:15:37,855][OSS][DEBUG] Measuring inference call with warmup: 3 and number: 1 and iterations 10
[2021-11-12 16:15:37,857][OSS][DEBUG] Warmup times: [0.0005461000837385654, 0.0005209916271269321, 0.0005161920562386513]


0.0005265118088573217

In [23]:
_, decoder_e2e_median_time = decoder_inference(
    t5_trt_decoder, input_ids, encoder_last_hidden_state, TimingProfile(iterations=10, number=1, warmup=1),
)
decoder_e2e_median_time

[2021-11-12 16:15:37,868][OSS][DEBUG] Measuring inference call with warmup: 3 and number: 1 and iterations 10
[2021-11-12 16:15:37,872][OSS][DEBUG] Warmup times: [0.0011899452656507492, 0.0010634041391313076, 0.0010606790892779827]


0.0010294795501977205

### Full model inference benchmark

Next, we will try the full TensorRT T5 engine for the task of translation. As before, note the time difference.

In [24]:
decoder_output_greedy, full_e2e_median_runtime = full_inference_greedy(
    t5_trt_encoder,
    t5_trt_decoder,
    input_ids,
    tokenizer,
    TimingProfile(iterations=10, number=1, warmup=1),
    max_length=T5ModelTRTConfig.MAX_SEQUENCE_LENGTH[metadata.variant],
    use_cuda=False,
)

print(tokenizer.decode(decoder_output_greedy[0], skip_special_tokens=True))
full_e2e_median_runtime


[2021-11-12 16:15:37,888][OSS][DEBUG] Measuring inference call with warmup: 3 and number: 1 and iterations 10
[2021-11-12 16:15:37,906][OSS][DEBUG] Warmup times: [0.006045436952263117, 0.005881193093955517, 0.00591813400387764]


Das ist gut.


0.005878558615222573

You can now compare the output of the original PyTorch model and the TensorRT engine. Notice the speed difference.

## Conclusion and where-to next?

This notebook has walked you through the process of converting a HuggingFace PyTorch T5 model to an optimized TensorRT engine for inference in 3 easy steps. The TensorRT inference engine can be conviniently used as a drop-in replacement for the orginial HuggingFace T5 model while providing significant speed up. 

Launch [t5-playground.ipynb](t5-playground.ipynb) for a text translation and summarization playground with T5.

If you are interested in further details of the conversion process, check out [T5/trt.py](../T5/trt.py)